In [139]:
import pandas as pd
import requests
import requests
from bs4 import BeautifulSoup
import time
import random
from io import StringIO
# temporary save to sqlite3
import sqlite3
import hashlib
from datetime import datetime

In [126]:
# from sqlalchemy import create_engine

# # Define your credentials
# db_user = "your_username"
# db_password = "your_password"
# db_host = "localhost"         # Or your RDS endpoint if hosted on AWS
# db_port = "5432"
# db_name = "your_db_name"

# # Use f-string to create the engine
# engine = create_engine(f"postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db_name}")

In [127]:
#Create headers to circumvent non-browser activity blocking (pd.read_html blocked)
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 "
                  "(KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"
}

#int'l tournaments without market-value data
blacklist_league_codes = {
    "CL", "EL", "UCOL", "CLI", "AFCL", "ACL", "CCL", "KLUB",
    "EM24", "EMQ", "CAM4", "WMQ4", "WM22", "AM23", "AFCN",
    "GC23", "19YL", "CLIY", "berater"
}

In [173]:
import sqlite3

def commit_changes(df, table_name, db = "transfermarkt.db"):
    conn = sqlite3.connect(db)

    # Save DataFrame to a new table (schema) named "players"
    df.to_sql(table, conn, if_exists="append", index=False)

    # Close connection
    conn.close()

def load_keys(table, primary_key, db = "transfermarkt.db"):
    conn = sqlite3.connect(db)

    # SQL query to fetch data
    query = f"SELECT DISTINCT {', '.join(primary_key)}, hash_key FROM {table}"

    # Load data into a DataFrame
    df = pd.read_sql_query(query, conn)

    # Close the connection
    conn.close()
    
    return df

def hash_row(row):
    row_string = ''.join(str(value) for value in row.values)
    return hashlib.sha256(row_string.encode()).hexdigest()


def get_soup(url):
    time.sleep(random.uniform(1, 5))
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return BeautifulSoup(response.text, "html.parser")
    else:
        print(f"Request failed with status: {response.status_code}")
        return None

def get_competitions():
    url = "https://www.transfermarkt.com/wettbewerbe/national"
    soup = get_soup(url)
    if not soup:
        return []

    competitions = []

    # Look for all <ul class="tm-button-list"> sections
    sections = soup.find_all("ul", class_="tm-button-list")
    for section in sections:
        links = section.find_all("a", class_="tm-button-list__list-item")
        for link in links:
            name = link.get("title")
            href = link.get("href")
            if name and href and 'agent' not in name.lower() and href.split("/")[-1] not in blacklist_league_codes:
                full_url = f"https://www.transfermarkt.com{href}"
                competitions.append((name, full_url))

    return competitions

def get_teams(url):
    print(url)
    response = requests.get(url, headers=headers)
    time.sleep(random.uniform(1, 5))
    
    if response.status_code == 200:
        
        try:
            data = pd.read_html(StringIO(response.text))
            if len(data) > 0:
                df = [i for i in data if 'total market value' in str(i).lower()][0]
                df["link"] = 'url'
                return df
        
        except:
            print(f"Table not found: {url}")
    else:
        print(f"Request failed with status code {response.status_code} ({url})")
    
    
    
# Example usage

history = load_keys('league', ['link'])

comp_data = get_competitions()
comps =  pd.DataFrame(comp_data, columns =  ['name', 'link'])
comps['hash_key'] = comps.apply(hash_row, axis=1)
comps = comps.drop_duplicates(subset = ["hash_key"]).reset_index(drop = True)

comps['effective_start_date'] = datetime.today().date()
comps['effective_end_date'] = pd.to_datetime('2099-12-31').date()
# comps.join

In [142]:
# import sqlite3

# # Connect to SQLite database (creates it if it doesn't exist)
# conn = sqlite3.connect("transfermarkt.db")

# # Save DataFrame to a new table (schema) named "players"
# comps.to_sql("league", conn, if_exists="replace", index=False)

# # Optional: verify schema creation
# print("Schema created and data saved successfully.")

# # Close connection
# conn.close()


Schema created and data saved successfully.


In [124]:
# from sqlalchemy import create_engine, Column, String, Integer, Float, Date, ForeignKey
# from sqlalchemy.ext.declarative import declarative_base

# Base = declarative_base()

# class League(Base):
#     __tablename__ = "league"
#     link = Column(String, primary_key=True)
#     name = Column(String)

C:\Users\kodri\AppData\Local\Temp\ipykernel_17180\728486554.py:4: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


### Teams

In [128]:
league_urls = [i[1] for i in comp_data]
dfs = [get_teams(url) for url in league_urls]

https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1
https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1
https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1
https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1
https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1
https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1
https://www.transfermarkt.com/laliga/startseite/wettbewerb/ES1
https://www.transfermarkt.com/serie-a/startseite/wettbewerb/IT1
https://www.transfermarkt.com/bundesliga/startseite/wettbewerb/L1
https://www.transfermarkt.com/ligue-1/startseite/wettbewerb/FR1
https://www.transfermarkt.com/liga-portugal/startseite/wettbewerb/PO1
https://www.transfermarkt.com/eredivisie/startseite/wettbewerb/NL1
https://www.transfermarkt.com/jupiler-pro-league/startseite/wettbewerb/BE1
https://www.transfermarkt.com/super-liga-srbije/startseite/wettbewerb/SER1
https://www.transfermarkt.com/super-league-1/startseite/w

In [97]:
for i in dfs:
    print(i.columns)

Index(['Club', 'Club.1', 'name', 'Squad', 'ø age', 'Foreigners',
       'ø market value', 'Total market value'],
      dtype='object')
Index(['Club', 'Club.1', 'name', 'Squad', 'ø age', 'Foreigners',
       'ø market value', 'Total market value'],
      dtype='object')
Index(['Club', 'Club.1', 'name', 'Squad', 'ø age', 'Foreigners',
       'ø market value', 'Total market value'],
      dtype='object')
Index(['Club', 'Club.1', 'name', 'Squad', 'ø age', 'Foreigners',
       'ø market value', 'Total market value'],
      dtype='object')
Index(['Club', 'Club.1', 'name', 'Squad', 'ø age', 'Foreigners',
       'ø market value', 'Total market value'],
      dtype='object')
Index(['Club', 'Club.1', 'name', 'Squad', 'ø age', 'Foreigners',
       'ø market value', 'Total market value'],
      dtype='object')
Index(['Club', 'Club.1', 'name', 'Squad', 'ø age', 'Foreigners',
       'ø market value', 'Total market value'],
      dtype='object')
Index(['Club', 'Club.1', 'name', 'Squad', 'ø age', 'For

In [14]:
def get_teams(competition_url):
    soup = get_soup(competition_url)
    if not soup:
        return []

    teams = []
    # Clubs are often listed under a table with class "items" on the competition page
    team_table = soup.find("table", class_="items")
    if not team_table:
        print(f"No team table found on {competition_url}")
        return []

    for row in team_table.find_all("tr", class_=["odd", "even"]):
        link_tag = row.find("a", class_="vereinprofil_tooltip")
        if link_tag:
            name = link_tag.get("title")
            href = link_tag.get("href")
            if name and href:
                full_url = f"https://www.transfermarkt.com{href}"
                teams.append((name, full_url))

    return teams

# Example usage
all_teams = []

for league, url in comp_data:
    print(url)
    teams = get_teams(url)
    break
#     for name, team_url in teams:
#         all_teams.append((league, name, team_url))

df_teams = pd.DataFrame(all_teams, columns=["league", "team", "team_url"])

https://www.transfermarkt.com/premier-league/startseite/wettbewerb/GB1


C:\Users\kodri\AppData\Local\Temp\ipykernel_17180\3570543701.py:13: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  dfs = pd.read_html(response.text)


In [28]:
counter = -1

### Players

In [ ]:
url = 'https://www.transfermarkt.com/arsenal-fc/startseite/verein/11'

#Create headers to circumvent non-browser activity blocking
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36'
}

response = requests.get(url, headers=headers)

#Read data to pandas
if response.status_code == 200:
    dfs = pd.read_html(response.text)
else:
    #Insert email notification or logging
    print(f"Request failed with status code {response.status_code}")


In [8]:
dfs[1]

,#,Player,Date of birth/Age,Nat.,Market value
0,22.0,David Raya Goalkeeper,"Sep 15, 1995 (29)",NaN,€40.00m
1,NaN,David Raya,NaN,NaN,NaN
2,NaN,Goalkeeper,NaN,NaN,NaN
3,32.0,Neto Goalkeeper,"Jul 19, 1989 (35)",NaN,€2.00m
4,NaN,Neto,NaN,NaN,NaN
...,...,...,...,...,...
67,NaN,Kai Havertz,NaN,NaN,NaN
68,NaN,Centre-Forward,NaN,NaN,NaN
69,9.0,Gabriel Jesus Centre-Forward,"Apr 3, 1997 (28)",NaN,€40.00m
70,NaN,Gabriel Jesus,NaN,NaN,NaN
